In [1]:
import pandas as pd
import pathlib
import numpy as np
from cemba_data.atac.dump_fragment_bed import read_snap_gene

In [2]:
gene_meta = pd.read_csv('/home/hanliu/ref/mouse/gencode/vm16/gencode.vM16.annotation.gene.flat.tsv.gz',
                        sep='\t', index_col='gene_name')
# this is no good......
gene_meta = gene_meta[~gene_meta.index.duplicated()].copy()


In [3]:
snap_dir = pathlib.Path('/home/hanliu/rdx4/CEMBA_ATAC/snap/')
snap_dict = {i.name.split('.')[0]: i for i in snap_dir.glob('*snap')}

In [4]:
cell_tidy_data = pd.read_csv('./rs1atac.cellLabel.meta.txt.gz', sep='\t')
brain_region = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/BrainRegion.txt', 
                           sep='\t', index_col='DissectionRegion')
cell_tidy_data.index = cell_tidy_data['sample'] + '-' + cell_tidy_data['barcode']

cell_tidy_data['CellClass'] = cell_tidy_data['L1'].copy()
cell_tidy_data['MajorType'] = cell_tidy_data['L2'].copy()
cell_tidy_data['SubType'] = cell_tidy_data['cluster'].copy()

cell_tidy_data['umap_0'] = cell_tidy_data['umap-1']
cell_tidy_data['umap_1'] = cell_tidy_data['umap-2']
cell_tidy_data['Region'] = cell_tidy_data['sample'].apply(lambda i: i.split('_')[1])
brain_region_table = cell_tidy_data['Region'].apply(lambda i: brain_region.loc[i])
brain_region_table.index = cell_tidy_data.index
cell_tidy_data = pd.concat([cell_tidy_data, brain_region_table], axis=1)

cell_tidy_data.to_csv('ATAC.cell_tidy_data.csv.gz')
cell_tidy_data.to_msgpack('ATAC.cell_tidy_data.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.


In [5]:
cell_tidy_data = pd.read_msgpack('ATAC.cell_tidy_data.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
for sample in cell_tidy_data['sample'].unique():
    if sample not in snap_dict:
        print(sample)

## Make adata

In [6]:
for sample, sub_df in cell_tidy_data.groupby('sample'):
    print(sample, sub_df.shape[0])
    if sample not in snap_dict:
        continue
    output_path = f'../Dataset_GM/{sample}.pass_qc.with_cluster.h5ad'
    
    if pathlib.Path(output_path).exists():
        continue
    
    snap_path = snap_dict[sample]
    adata = read_snap_gene(snap_path)
    adata.obs_names = sample + '-' + adata.obs['barcode']
    adata = adata[sub_df.index, :].copy()
    adata.obs.update(sub_df)
    adata.obs = pd.concat([
        adata.obs,
        sub_df[[i for i in sub_df.columns if i not in adata.obs.columns]]
    ],
                          axis=1)
    
    adata.var = gene_meta.reindex(adata.var_names)
    adata.var['gene_name'] = adata.var_names
    adata.var_names = adata.var['gene_id']
    adata.write_h5ad(output_path)

CEMBA171206_3C 9207


/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
... storing 'sample' as categorical
... storing 'cluster' as categorical
... storing 'L1' as categorical
... storing 'L2' as categorical
... storing 'L3' as categorical
... storing 'CellClass' as categorical
... storing 'MajorType' as categorical
... storing 'SubType' as categorical
... storing 'chrom' as categorical
... storing 'source' as categorical
... storing 'feature' as categorical
... storing 'score' as categorical
... storing 'strand' as categorical
... storing 'phase' as categorical
... storing 'gene_type' as categorical
... storing 'havana_gene' as categorical
... storing 'tag' as categorical


CEMBA171207_3C 8961


... storing 'sample' as categorical
... storing 'cluster' as categorical
... storing 'L1' as categorical
... storing 'L2' as categorical
... storing 'L3' as categorical
... storing 'CellClass' as categorical
... storing 'MajorType' as categorical
... storing 'SubType' as categorical
... storing 'chrom' as categorical
... storing 'source' as categorical
... storing 'feature' as categorical
... storing 'score' as categorical
... storing 'strand' as categorical
... storing 'phase' as categorical
... storing 'gene_type' as categorical
... storing 'havana_gene' as categorical
... storing 'tag' as categorical


CEMBA171213_4B 9048


...

... storing 'sample' as categorical
... storing 'cluster' as categorical
... storing 'L1' as categorical
... storing 'L2' as categorical
... storing 'L3' as categorical
... storing 'CellClass' as categorical
... storing 'MajorType' as categorical
... storing 'SubType' as categorical
... storing 'chrom' as categorical
... storing 'source' as categorical
... storing 'feature' as categorical
... storing 'score' as categorical
... storing 'strand' as categorical
... storing 'phase' as categorical
... storing 'gene_type' as categorical
... storing 'havana_gene' as categorical
... storing 'tag' as categorical


CEMBA190716_8E 11058


... storing 'sample' as categorical
... storing 'cluster' as categorical
... storing 'L1' as categorical
... storing 'L2' as categorical
... storing 'L3' as categorical
... storing 'CellClass' as categorical
... storing 'MajorType' as categorical
... storing 'SubType' as categorical
... storing 'chrom' as categorical
... storing 'source' as categorical
... storing 'feature' as categorical
... storing 'score' as categorical
... storing 'strand' as categorical
... storing 'phase' as categorical
... storing 'gene_type' as categorical
... storing 'havana_gene' as categorical
... storing 'tag' as categorical


CEMBA190716_8J 11936


... storing 'sample' as categorical
... storing 'cluster' as categorical
... storing 'L1' as categorical
... storing 'L2' as categorical
... storing 'L3' as categorical
... storing 'CellClass' as categorical
... storing 'MajorType' as categorical
... storing 'SubType' as categorical
... storing 'chrom' as categorical
... storing 'source' as categorical
... storing 'feature' as categorical
... storing 'score' as categorical
... storing 'strand' as categorical
... storing 'phase' as categorical
... storing 'gene_type' as categorical
... storing 'havana_gene' as categorical
... storing 'tag' as categorical
